# Efficient Decompositions of 2-Qubit Unitaries
Implementing both B-gate and SQiSW-based methods of 2-Qubit quantum gate decomposition.

Kerstin Fagerstrom and Thomas Verrill

ECE 396: Quantum Computing

December 17, 2022

In [ ]:
# initialize the notebook and necessary packages
!pip install qiskit ipywidgets pylatexenc

'''Some generic python and jupyter imports'''
import numpy as np
from numpy import pi,sqrt

''' Some Jupyter notebook settings'''
%matplotlib inline
%config InlineBackend.figure_format ='retina'

'''Qiskit imports'''
from qiskit import(
    QuantumCircuit
    , execute
    , Aer
    , ClassicalRegister
    , QuantumRegister
)

#more qiskit and numpy imports
backend_qasm = Aer.get_backend('qasm_simulator')
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import TwoQubitBasisDecomposer
from qiskit.quantum_info import random_unitary
from qiskit.circuit.library import CXGate
from qiskit.compiler import transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import process_fidelity
from qiskit.extensions import RXGate, XGate, CXGate
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info.synthesis.two_qubit_decompose import TwoQubitWeylDecomposition
from numpy.linalg.linalg import transpose
from numpy.linalg import *
from qiskit.quantum_info.synthesis import OneQubitEulerDecomposer
from qiskit import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.9/929.9 KB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━

# B Gate Decomposition
Implements B gate decompostion as discussed in "Minimum construction of two-qubit quantum operations."

###Random Unitary Initialization

In [ ]:
# initialize random unitary to decompose
ru = random_unitary(4)

In [ ]:
# Find Weyl Decomposition coordinates of arbitrary unitary
a = TwoQubitWeylDecomposition(ru).a*2 #multiplication by 2 because Qiskit scales coordinates by 1/2
b = TwoQubitWeylDecomposition(ru).b*2
c = TwoQubitWeylDecomposition(ru).c*2

### B gate

In [ ]:
#b_gate implementation as defined in the research paper
b_gate = QuantumCircuit(2)
b_gate.rxx(-pi/2, 0,1)
b_gate.ryy(-pi/4,0,1)
B = Operator(b_gate)

### Implement B gate sandwich

In [ ]:
# constants to implement the nonlocal operation A (the B sandwich)
beta1 = np.arccos(1-4*((np.sin(b/2))**2)*((np.cos(c/2))**2))
beta2 = np.arcsin(sqrt((np.cos(b)*np.cos(c))/(1-2*(np.sin(b/2))**2*(np.cos(c/2))**2)))

In [ ]:
# initialize decomposition circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)

# non local operation A (B gates with rotations in between based on constants)
def B_sand(temp_circ):
  temp_circ.append(B, qr)
  temp_circ.ry(-a,qr[0])
  temp_circ.rz(-beta2,qr[1])
  temp_circ.ry(-beta1,qr[1])
  temp_circ.rz(-beta2,qr[1])
  temp_circ.append(B, qr)

#implement B_sandwich into circuit
B_sand(circ)
#create B sandwich operator
o_c = Operator(circ)

In [ ]:
# Find Weyl Decomposition coordinates of B sandwich 
a1 = TwoQubitWeylDecomposition(o_c).a*2
b1 = TwoQubitWeylDecomposition(o_c).b*2
c1 = TwoQubitWeylDecomposition(o_c).c*2

In [ ]:
# phase correction
# fixes the circuit in the case that the c coordinates of unitary and sandwich are opposite signs  
if(np.sign(c1) != np.sign(c)): 
  qr1 = QuantumRegister(2)
  new_circ = QuantumCircuit(qr1)

  #new B_sandwich (sign of a on ry rotation is flipped)
  new_circ.append(B, qr1)
  new_circ.ry(a,qr1[0])
  new_circ.rz(-beta2,qr1[1])
  new_circ.ry(-beta1,qr1[1])
  new_circ.rz(-beta2,qr1[1])
  new_circ.append(B, qr1)

  #implement B_sandwich into circuit
  circ = new_circ
  #create B sandwich operator
  o_c = Operator(circ)

### Final Circuit & Results

In [ ]:
#Implement pre/post local rotations to B sandwich

#initialize Weyl Decomposition of B_sandwich and random unitary
b_decomp = TwoQubitWeylDecomposition(o_c)
u_decomp = TwoQubitWeylDecomposition(ru)

#create circuit for pre/post local rotations 
new_circuit = QuantumCircuit(2)

#add the local random unitary pre-rotations to B sandwich
new_circuit.append(Operator(u_decomp.K2r), [0])  
new_circuit.append(Operator(u_decomp.K2l), [1])

#add inverses for the local Bsandwich pre-rotations
new_circuit.append(Operator(inv(b_decomp.K2r)), [0])   
new_circuit.append(Operator(inv(b_decomp.K2l)), [1])

#add the B sandwich
new_circuit.append(o_c, [0,1])

#add inverses for the local B_sandwich post-rotations
new_circuit.append(Operator(inv(b_decomp.K1r)), [0])  
new_circuit.append(Operator(inv(b_decomp.K1l)), [1])

#add the local random unitary post-rotations to B sandwich
new_circuit.append(Operator(u_decomp.K1r), [0])   
new_circuit.append(Operator(u_decomp.K1l), [1])

In [ ]:
# build final circuit and calculate/implement global phase
final_circ = new_circuit
final_circ.global_phase = u_decomp.global_phase
#create operator for the final circuit
final_circ = Operator(final_circ)

In [ ]:
# Find Weyl Decomposition coordinates of the final circuit
a2 = TwoQubitWeylDecomposition(final_circ).a*2
b2 = TwoQubitWeylDecomposition(final_circ).b*2
c2 = TwoQubitWeylDecomposition(final_circ).c*2

In [ ]:
# results
# compare the Weyl coordinates of the decomposition with those of the unitary
# if the coordinates are the same, the operators are locally equivalent
print("Weyl Coordinates:")
print("   decomposition     original")
print("a: " + str(a2) + " " + str(a))
print("b: " + str(b2) + "  " + str(b))
print("c: " + str(c2) + " " + str(c))
#compare the operators themselves
#if "compare" returns true, the decomposition is complete
compare = Operator(final_circ) == Operator(ru)
print("The B-decomposition is the same operator as the random unitary: " + str(compare))
#create Weyl Decomposition of the final circuit to compare with the unitary circuit Weyl decomposition
final_decomp = TwoQubitWeylDecomposition(final_circ)
#print out each Weyl decomposition circuit and global phase (they should be equal)
print("Unitary Weyl decomposition circuit and global phase:")
print(u_decomp)
print("B-decomposition Weyl decomposition circuit and global phase:")
print(final_decomp)

Weyl Coordinates:
   decomposition     original
a: 1.3515503288522526 1.3515503288522526
b: 0.7006073148596563  0.7006073148596563
c: -0.25128150790708936 -0.25128150790708936
The B-decomposition is the same operator as the random unitary: True
Unitary Weyl decomposition circuit and global phase:
TwoQubitWeylGeneral(
	global phase: 1.3744
	     ┌───────────────┐┌────────────┐┌────────────┐┌───────────────┐┌────────────────┐               ┌────────────┐┌─────────────┐┌─────────────┐
	q_0: ┤ Rz(-0.086884) ├┤ Ry(1.1459) ├┤ Rz(1.6039) ├┤0              ├┤0               ├─■─────────────┤ Rz(2.6992) ├┤ Ry(0.78438) ├┤ Rz(-2.8822) ├
	     └┬─────────────┬┘├────────────┤├────────────┤│  Rxx(-1.3516) ││  Ryy(-0.70061) │ │ZZ(0.25128) ┌┴────────────┤└┬────────────┤├─────────────┤
	q_1: ─┤ Rz(-2.9409) ├─┤ Ry(2.6121) ├┤ Rz(1.9817) ├┤1              ├┤1               ├─■────────────┤ Rz(0.19848) ├─┤ Ry(1.8124) ├┤ Rz(-2.7774) ├
	      └─────────────┘ └────────────┘└────────────┘└───────────────┘└──────

# SWiSQ Decomposition
Implements SWiSQ based decomposition as discussed in "Quantum Instruction Set Design for Performance" Algorithm 1.

### Implement SQiSW Gate

In [ ]:
#sqrt(iswap) (abbreviated as sis) implementation
sqrt2 = np.sqrt(2)
#defining sqrt(iswap) operator
sis = Operator( [
    [1, 0, 0, 0],
    [0, 1/sqrt2, 1j/sqrt2, 0],
    [0, 1j/sqrt2, 1/sqrt2, 0],
    [0, 0, 0, 1] ]) 

### Interleaving Single Qubit Gates Method
(From Algorithm 1 in the paper)






In [ ]:
# "Output the single qubit rotations given the interaction coefficients (x, y, z) ∈ W ′ when sandwiched by two SQiSW gates" 
# a, b, c are Weyl Decomposition Coordinates (no scaling needed)
def InterleavingSingleQubitGates(a,b,c):
  C = np.sin(a + b - c)*np.sin(a - b + c)*np.sin(-a - b - c)*np.sin(-a + b + c)
  alpha = np.arccos(np.cos(2*a)-np.cos(2*b)+ np.cos(2*c)+ 2*sqrt(C))
  beta = np.arccos(np.cos(2*a)-np.cos(2*b)+ np.cos(2*c) - 2*sqrt(C))
  gamma = np.arccos(np.sign(c) * sqrt((4*np.cos(a)**2*np.cos(c)**2*np.sin(b)**2)/
                                      (4*np.cos(a)**2*np.cos(c)**2*np.sin(b)**2 + np.cos(2*a)*np.cos(2*b)*np.cos(2*c))))
  
  c1_circ = QuantumCircuit(1)
  c2_circ = QuantumCircuit(1)
  c1_circ.rz(gamma,0)
  c1_circ.rx(alpha,0)
  c1_circ.rz(gamma,0)
  c2_circ.rx(beta,0)

  #the returned operators correspond to C1 and C2 in the paper
  return Operator(c1_circ), Operator(c2_circ) 

#test the function with sample Weyl coordinates
print(InterleavingSingleQubitGates(pi/4,pi/8,0))

(Operator([[2.22044605e-16-1.j, 0.00000000e+00+0.j],
          [0.00000000e+00+0.j, 2.22044605e-16+1.j]],
         input_dims=(2,), output_dims=(2,)), Operator([[0.5411961+0.j        , 0.       -0.84089642j],
          [0.       -0.84089642j, 0.5411961+0.j        ]],
         input_dims=(2,), output_dims=(2,)))


### Canonicalize Method
(From Algorithm 1 in the paper)

In [ ]:
# "Decompose an arbitrary gate into one SQiSW and one L(x′, y′, z′) 
# where (x′, y′, z′) ∈ W ′ and output the coefficients (x′, y′, z′) 
# and the interleaving single qubit rotations"
def Canonicalize(a,b,c):
  A1 = QuantumCircuit(1)
  A2 = QuantumCircuit(1)
  B1 = QuantumCircuit(1)
  B1.ry(-pi/2,0)
  B2 = QuantumCircuit(1)
  B2.ry(pi/2,0)
  C1 = QuantumCircuit(1)
  C1.ry(pi/2,0)
  C2 = QuantumCircuit(1)
  C2.ry(-pi/2,0)

  s = np.sign(c)

  #a_ corresponds to a' in the paper, and so on
  a_ = a
  b_ = b
  c_ = abs(c)

  if (a > pi/8):
    b_ -= pi/8
    c_ -= pi/8
    B1.rz(pi/2,0)
    B2.rz(-pi/2,0)
    saved_C1 = Operator(C1)
    C1 = QuantumCircuit(1)
    C1.rz(-pi/2,0)
    C1.append(saved_C1,[0])
    saved_C2 = Operator(C2)
    C2 = QuantumCircuit(1)
    C2.rz(pi/2,0)
    C2.append(saved_C2,[0])
  else:
    a_ += pi/8
    c_ -= pi/8
  if (abs(b_)<abs(c_)):
    c_ = -c_
    #b_ = -b_
    A1.rx(pi/2, 0)
    A2.rx(-pi/2,0)
    B1.rx(-pi/2,0)
    B2.rx(pi/2,0)
  if (s<0):
    c_ = -c_
    saved_A1 = Operator(A1)
    saved_B1 = Operator(B1)
    saved_C1 = Operator(C1)
    A1 = QuantumCircuit(1)
    B1 = QuantumCircuit(1)
    C1 = QuantumCircuit(1)
    A1.z(0)
    A1.append(saved_A1,[0])
    A1.z(0)
    B1.z(0)
    B1.append(saved_B1,[0])
    B1.z(0)
    C1.z(0)
    C1.append(saved_C1,[0])
    C1.z(0)

  return a_,b_,c_, Operator(A1), Operator(A2), Operator(B1), Operator(B2), Operator(C1), Operator(C2)

#test the method with sample Weyl Coordinates
print(Canonicalize(pi/4,pi/8,0))

(0.7853981633974483, 0.0, 0.39269908169872414, Operator([[0.70710678+0.j        , 0.        -0.70710678j],
          [0.        -0.70710678j, 0.70710678+0.j        ]],
         input_dims=(2,), output_dims=(2,)), Operator([[0.70710678+0.j        , 0.        +0.70710678j],
          [0.        +0.70710678j, 0.70710678+0.j        ]],
         input_dims=(2,), output_dims=(2,)), Operator([[ 7.07106781e-01-7.07106781e-01j,
            5.55111512e-17+5.55111512e-17j],
          [-5.55111512e-17+5.55111512e-17j,
            7.07106781e-01+7.07106781e-01j]],
         input_dims=(2,), output_dims=(2,)), Operator([[ 1.66533454e-16+5.55111512e-17j,
           -7.07106781e-01-7.07106781e-01j],
          [ 7.07106781e-01-7.07106781e-01j,
            1.66533454e-16-5.55111512e-17j]],
         input_dims=(2,), output_dims=(2,)), Operator([[ 0.5+0.5j, -0.5+0.5j],
          [ 0.5+0.5j,  0.5-0.5j]],
         input_dims=(2,), output_dims=(2,)), Operator([[ 0.5-0.5j,  0.5+0.5j],
          [-0.5+0.5j,  0.

###KAK Decomposition Method 
(as it relates to Algorithm 1 from the paper)

In [ ]:
# decompose the circuit such that "U = g · (A1 ⊗ A2)L(x, y, z)(B1 ⊗ B2)"
# we use the pre/post rotation decomposition as with the B-gate decomposition method
def KAKDecomp(u):
  u_decomp = TwoQubitWeylDecomposition(u)
  #global phase
  g = u_decomp.global_phase
  #Weyl coordinates of unitary
  a_u = u_decomp.a
  b_u = u_decomp.b
  c_u = u_decomp.c
  #pre/post rotations
  A1 = u_decomp.K1r
  A2 = u_decomp.K1l
  B1 = u_decomp.K2r
  B2 = u_decomp.K2l

  return g, a_u, b_u, c_u, A1, A2, B1, B2

#test the KAKDecomp method with a random 2-qubit unitary
print(KAKDecomp(random_unitary(4)))

(1.6804884970047922, 0.749986920079829, 0.4239620842951212, 0.2172831325354171, array([[-0.08525431-0.08154287j,  0.74200064-0.6599375j ],
       [-0.74200064-0.6599375j , -0.08525431+0.08154287j]]), array([[ 0.13871107-0.5676728j , -0.11529743+0.80325172j],
       [ 0.11529743+0.80325172j,  0.13871107+0.5676728j ]]), array([[ 0.31974121-0.11901251j, -0.93773644-0.06520703j],
       [ 0.93773644-0.06520703j,  0.31974121+0.11901251j]]), array([[ 0.50146016+0.1810523j ,  0.8332392 +0.14652712j],
       [-0.8332392 +0.14652712j,  0.50146016-0.1810523j ]]))


### Full Decomposition Method
(Decomp(U) from the paper)

In [ ]:
#"Decompose U into single qubit gates and the SQiSW gate"
#u is a random 2 qubit unitary
def sis_decomp(u):
  g, a, b, c, A1, A2, B1, B2 = KAKDecomp(u)
  A1 = Operator(A1)
  A2 = Operator(A2)
  B1 = Operator(B1)
  B2 = Operator(B2)

  #in the case that two sis gates are needed
  if (abs(c) <= (a-b)):
    C1, C2 = InterleavingSingleQubitGates(a,b,c)

    # create V operator
    V = QuantumCircuit(2)
    V.append(sis,[0,1])
    V.append(C1,[0])
    V.append(C2,[1])
    V.append(sis,[0,1])
    V = Operator(V)

    g3, a3, b3, c3, D1, D2, E1, E2= KAKDecomp(V)

    # calculate inverses from KAKDecomp(V)
    D1i = Operator(inv(D1))
    D2i = Operator(inv(D2))
    E1i = Operator(inv(E1))
    E2i = Operator(inv(E2))

    # create decomposed circuit to be returned
    ret_c = QuantumCircuit(2)

    ret_c.append(B1, [0])
    ret_c.append(B2, [1])
    ret_c.append(E1i, [0])
    ret_c.append(E2i, [1])

    ret_c.append(sis, [0,1])

    ret_c.append(C1, [0])
    ret_c.append(C2, [1])

    ret_c.append(sis, [0,1])

    ret_c.append(D1i, [0])
    ret_c.append(D2i, [1])
    ret_c.append(A1, [0])
    ret_c.append(A2, [1])

    return ret_c

  #in the case that three sis gates are needed
  else:
    a_,b_,c_, F1, F2, G1, G2, H1, H2 = Canonicalize(a,b,c)
    C1, C2 = InterleavingSingleQubitGates(a_,b_,c_)

    # create V operator
    V = QuantumCircuit(2)
    V.append(sis,[0,1])
    V.append(C1,[0])
    V.append(C2,[1])
    V.append(sis,[0,1])
    V = Operator(V)
    
    g3, a3, b3, c3, D1, D2, E1, E2= KAKDecomp(V)

    # calculate inverses from KAKDecomp(V)
    D1i = Operator(inv(D1))
    D2i = Operator(inv(D2))
    E1i = Operator(inv(E1))
    E2i = Operator(inv(E2))

    # create decomposed return circuit
    ret_c = QuantumCircuit(2)

    ret_c.append(B1, [0])
    ret_c.append(B2, [1])
    ret_c.append(H1, [0])
    ret_c.append(H2, [1])

    ret_c.append(sis, [0,1])

    ret_c.append(G1, [0])
    ret_c.append(G2, [1])
    ret_c.append(E1i, [0])
    ret_c.append(E2i, [1])

    ret_c.append(sis, [0,1])

    ret_c.append(C1, [0])
    ret_c.append(C2, [1])

    ret_c.append(sis, [0,1])

    ret_c.append(D1i, [0])
    ret_c.append(D2i, [1])
    ret_c.append(F1, [0])
    ret_c.append(F2, [1])
    ret_c.append(A1, [0])
    ret_c.append(A2, [1])

    return ret_c

### Final Circuit & Results

In [ ]:
# initialize final circuit and random unitary
cxc = QuantumCircuit(2)
cxc.cx(0,1)
cxc = Operator(cxc)
u10 = cxc #random_unitary(4)
# create Weyl Decomposition circuit for random unitary
u10_decomp = TwoQubitWeylDecomposition(u10)
# decompose unitary with Algorithm 1 from paper
new_c = sis_decomp(u10)
#print out the decomposed circuit
print(new_c)

     ┌─────────┐┌─────────┐┌──────────┐┌─────────┐┌──────────┐┌─────────┐»
q_0: ┤ Unitary ├┤ Unitary ├┤0         ├┤ Unitary ├┤0         ├┤ Unitary ├»
     ├─────────┤├─────────┤│  Unitary │├─────────┤│  Unitary │├─────────┤»
q_1: ┤ Unitary ├┤ Unitary ├┤1         ├┤ Unitary ├┤1         ├┤ Unitary ├»
     └─────────┘└─────────┘└──────────┘└─────────┘└──────────┘└─────────┘»
«     ┌─────────┐
«q_0: ┤ Unitary ├
«     ├─────────┤
«q_1: ┤ Unitary ├
«     └─────────┘


####Apply local pre and post rotations to decomposition circuit

In [ ]:
#create operator from decomposition circuit and find Weyl Decomposition
new_c_op = Operator(new_c)
new_c_decomp = TwoQubitWeylDecomposition(new_c_op)

#create temporary circuit for the pre and post rotations
temp_circuit1 = QuantumCircuit(2)

#apply pre/post rotations based on method used for B-gate decomposition
# temp_circuit1.append(Operator(u10_decomp.K2r), [0])  
# temp_circuit1.append(Operator(u10_decomp.K2l), [1])

# temp_circuit1.append(Operator(inv(new_c_decomp.K2r)), [0])   
# temp_circuit1.append(Operator(inv(new_c_decomp.K2l)), [1])

temp_circuit1.append(new_c_op, [0,1])

# temp_circuit1.append(Operator(inv(new_c_decomp.K1r)), [0])  
# temp_circuit1.append(Operator(inv(new_c_decomp.K1l)), [1])

# temp_circuit1.append(Operator(u10_decomp.K1r), [0])   
# temp_circuit1.append(Operator(u10_decomp.K1l), [1])

In [ ]:
# construct final circuit with global phase
final_circ1 = temp_circuit1
final_circ1.global_phase = u10_decomp.global_phase

#create operator from final circuit to find Weyl decomposition
final_circ1 = Operator(final_circ1)
final_c_decomp = TwoQubitWeylDecomposition(final_circ1)

In [ ]:
# test and print results
# find weyl coordinates of the random unitary and the final decomposed circuit
test_a = u10_decomp.a
test_b = u10_decomp.b
test_c = u10_decomp.c
circ_a = final_c_decomp.a
circ_b = final_c_decomp.b
circ_c = final_c_decomp.c

# compare the Weyl coordinates of the decomposition with those of the unitary
# if the coordinates are the same, the operators are locally equivalent
print("Weyl Coordinates:")
print("   decomposition     original")
print("a: " + str(circ_a) + " " + str(test_a))
print("b: " + str(circ_b) + "  " + str(test_b))
print("c: " + str(circ_c) + " " + str(test_c))
#compare the operators themselves
#if "compare" returns true, the decomposition is complete
compare1 = Operator(final_circ1) == Operator(u10)
print("The sis decomposition is the same operator as the random unitary: " + str(compare1))

#print out the Weyl decomposition circuits of the unitary and the decomposed circuit
#(they should be the same circuit)

print("sis decomposition:")
print(final_c_decomp)
print("unitary:")
print(u10_decomp)

Weyl Coordinates:
   decomposition     original
a: 0.7853981633974483 0.7853981633974483
b: 0  0
c: 0 0
The sis decomposition is the same operator as the random unitary: False
sis decomposition:
TwoQubitWeylControlledEquiv(
	global phase: 5π/4
	      ┌──────────┐ ┌────────┐┌────────────┐ ┌──────────┐ ┌─────────┐
	q_0: ─┤ Ry(-π/2) ├─┤ Rx(-π) ├┤0           ├─┤ Rx(-π/2) ├─┤ Ry(π/2) ├
	     ┌┴──────────┴┐└────────┘│  Rxx(-π/2) │┌┴──────────┴┐└─────────┘
	q_1: ┤ Rx(2.2516) ├──────────┤1           ├┤ Rx(2.4608) ├───────────
	     └────────────┘          └────────────┘└────────────┘           
)
unitary:
TwoQubitWeylControlledEquiv(
	global phase: π/4
	      ┌─────────┐  ┌────────────┐   ┌─────────┐   ┌──────────┐
	q_0: ─┤ Ry(π/2) ├──┤0           ├───┤ Rx(π/2) ├───┤ Ry(-π/2) ├
	     ┌┴─────────┴─┐│  Rxx(-π/2) │┌──┴─────────┴──┐└──────────┘
	q_1: ┤ Rx(1.5695) ├┤1           ├┤ Rx(0.0013233) ├────────────
	     └────────────┘└────────────┘└───────────────┘            
)
